<a href="https://colab.research.google.com/github/aekansh-08/CSOC-dev-week2/blob/master/cnn_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.utils import np_utils



Using TensorFlow backend.


In [10]:
pip install wandb

     |████████████████████████████████| 1.4MB 2.5MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
     |████████████████████████████████| 102kB 8.7MB/s 
     |████████████████████████████████| 460kB 11.4MB/s 
     |████████████████████████████████| 112kB 20.0MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=69880107d84dc84bf011bc9aad5ac4ac7842898f5b18691d0ac27518feaa6869
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=b249a757af66f8c2476c0271a42f88425bbbea26fc7112beedcb7cc5a75b6b00
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=b9fd

In [0]:

from wandb.keras import WandbCallback
import wandb
import os


In [12]:
run = wandb.init()
config = run.config
config.first_layer_convs = 32
config.first_layer_conv_width = 3
config.first_layer_conv_height = 3
config.dropout = 0.2
config.dense_layer_size = 100
config.img_width = 28
config.img_height = 28
config.epochs = 50


<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [13]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [0]:
X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

In [15]:
X_train.shape

(60000, 28, 28)

In [0]:
X_train = X_train.reshape(
    X_train.shape[0], config.img_width, config.img_height, 1)
X_test = X_test.reshape(
    X_test.shape[0], config.img_width, config.img_height, 1)

In [17]:
X_train.shape

(60000, 28, 28, 1)

In [0]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
labels = range(10)


In [19]:
y_train.shape

(60000, 10)

In [20]:
y_test.shape

(10000, 10)

In [21]:
model = Sequential()
model.add(Conv2D(32,
                 (config.first_layer_conv_width, config.first_layer_conv_height),
                 input_shape=(28, 28, 1),
                 activation='relu')) #3x3 is the size of kernel and 1 bias for each kernel and we are putting 32 layers so the no of parameters are(32((3x3)+1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Conv2D(64,
                 (3, 3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(config.dense_layer_size, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)             

In [22]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),
          epochs=config.epochs,
          callbacks=[WandbCallback(data_type="image", save_model=False)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 23s 381us/step - loss: 0.2842 - accuracy: 0.9101 - val_loss: 0.0550 - val_accuracy: 0.9826
Epoch 2/50
60000/60000 [==============================] - 17s 279us/step - loss: 0.1151 - accuracy: 0.9658 - val_loss: 0.0434 - val_accuracy: 0.9853
Epoch 3/50
60000/60000 [==============================] - 17s 281us/step - loss: 0.0930 - accuracy: 0.9722 - val_loss: 0.0326 - val_accuracy: 0.9891
Epoch 4/50
60000/60000 [==============================] - 17s 280us/step - loss: 0.0776 - accuracy: 0.9762 - val_loss: 0.0295 - val_accuracy: 0.9901
Epoch 5/50
60000/60000 [==============================] - 17s 281us/step - loss: 0.0712 - accuracy: 0.9790 - val_loss: 0.0270 - val_accuracy: 0.9910
Epoch 6/50
60000/60000 [==============================] - 17s 278us/step - loss: 0.0643 - accuracy: 0.9808 - val_loss: 0.0280 - val_accuracy: 0.9912
Epoch 7/50
60000/60000 [==============================] 